# Counting Gitcoin From the Outside: A Short Analysis of Gitcoin Round 4

## About this Notebook

- This is contains some analyis of data we collected using only (1) the Gitcoin API and (2) etherscan. It represents our current best efforts to replicate a "true" dataset of Gitcoin contributions. We hope this will be useful for anyone generally interested in Gitcoin (e.g. crowdfunding researchers, curious potential Gitcoin users), but we are particularly interested in studying the production functions in past Gitcoin rounds.
- Our approach still has some issues, and we'd for anyone interested (perhaps you, the reader) to weigh in with how to improve it. The core issue is that we haven't yet figured out how to filter out transactions made during a certain round (say, round 4) that are sent *to* a grant address from a later round. So we end up including some transactions that do not appear in the official Gitcoin blog posts.
- (Part of the challenge is that we have round-specific funder addresses, but a global list of project/recipient addresses)

On with the code! Imports to start: we'll use just pandas and numpy.

In [1]:
import pandas as pd
import numpy as np
import json
import requests


Load the processed data about each Gitcoin rounds.
The processing is performed in `process_rounds_data.ipynb` and saved in `nice_rounds.json`

In [2]:
with open('nice_rounds.json', 'r') as f:
    nice_rounds = json.load(f)

Load a set of scraped observations. Each row is one ETH or ERC20 transaction to or from the set of Gitcoin contributors. 

In [3]:
df = pd.read_csv('transactions/4_transactions.csv', index_col=0)

Review the columns available.

In [4]:
df.columns

Index(['blockNumber', 'timeStamp', 'hash', 'nonce', 'blockHash',
       'transactionIndex', 'from', 'to', 'value', 'gas', 'gasPrice', 'isError',
       'txreceipt_status', 'input', 'contractAddress', 'cumulativeGasUsed',
       'gasUsed', 'confirmations', 'tokenName', 'tokenSymbol', 'tokenDecimal'],
      dtype='object')

How many transactions total?

In [5]:
len(df)

75049

Get latest data about all active gitcoin grants. We will only include transactions sent TO a gitcoin grant address. Note that the slightly more accurate version will be to get the round specific grant info.

In [6]:
gitcoin_resp = requests.get('https://gitcoin.co/grants/grants.json')
gitcoin_data = gitcoin_resp.json()

Keep a dict that maps GRANT ADDRESS to GRANT NAME.

Also make sure all addresses use only lowercase for string matching.

In [7]:
address_to_name = {}
for name, address in gitcoin_data:
    address_to_name[address.lower()] = name

Find all observations that are TO a grant address (so we can drop all other observations)

In [12]:
addresses = list(address_to_name.keys())
# only want rows that corresponds to ETH or ERT20 that "is sent to project"
is_sent_to_project_mask = df['to'].str.lower().isin(addresses)
df_filtered = df[is_sent_to_project_mask].copy()

In [13]:
len(df_filtered)

8125

See total counts for our ERC 20 tokens and ETH transactions (tokenName = None implies a transaction is in ETH)

In [14]:
df_filtered.loc[df_filtered.tokenName.isna(), 'tokenName'] = 'ETH'

In [18]:
df_filtered.tokenName.value_counts()[:20]

Dai Stablecoin                          6183
ETH                                      253
Sai Stablecoin v1.0                      175
Wrapped Ether                            114
Uniswap V1                                94
Synth sUSD                                87
Synthetix Network Token                   86
USD Coin                                  82
Aave Interest bearing DAI                 75
Chai                                      58
KyberNetwork                              55
Redeemable DAI                            40
Axie                                      33
Aave Interest bearing ETH                 32
ChainLink Token                           26
Fulcrum Perpetual Long ETH-DAI 2x v2      25
Maker                                     24
SAN                                       22
DAOstack                                  21
Compound Dai                              21
Name: tokenName, dtype: int64

Let's keep only ETH and DAI, and call the version with only these transactions `df_a` (version 'a', with the possibility of creating versions b, c, ... later)

In [ ]:
df_filtered.loc[df_filtered.tokenName == 'Wrapped Ether', tokenName] = 'ETH'

Note: for now, we're going to uncritically assume that wrapped ether is just ETH
and that all the stablecoins convert directly to 1 USD

In [19]:
df_filtered

,blockNumber,timeStamp,hash,nonce,blockHash,transactionIndex,from,to,value,gas,...,isError,txreceipt_status,input,contractAddress,cumulativeGasUsed,gasUsed,confirmations,tokenName,tokenSymbol,tokenDecimal
4,9272432,1578915412,0x9e49c1e0631b685ae2a2cbd1dc32e1e521729f92e661...,153,0x17f36b1282af804873b6fabcde9e1f221f4877403905...,83,0x00158a74921620b39e5c3afe4dca79feb2c2c143,0x322d58b9e75a6918f7e7849aee0ff09369977e08,4000000000000000000,100000,...,NaN,NaN,deprecated,0x6b175474e89094c44da98b954eedeac495271d0f,7217413,46008,4308028,Dai Stablecoin,DAI,18.0
52,9243147,1578528442,0x70518de1df557d61d92d3b7c1d5394fd264c634f54b4...,215,0xfc648bcb632d8f93255849f86be531415f25a6653fd9...,30,0x002781a27becd321d831d815e3fa386944b329c8,0xbd9cdcb6688b057080b1767261c5dc17a5804e82,23750000000000000000,100000,...,NaN,NaN,deprecated,0x6b175474e89094c44da98b954eedeac495271d0f,2992428,50292,4337313,Dai Stablecoin,DAI,18.0
57,9244538,1578546802,0xd884392de6460d8937532021e5b39e4f1e4e8d2506a7...,220,0x1c9c02ef00980297e40a30c370cafa83eeb058139e87...,65,0x002781a27becd321d831d815e3fa386944b329c8,0x865c2f85c9fea1c6ac7f53de07554d68cb92ed88,1000000000000000000,78057,...,NaN,NaN,deprecated,0x892091bdd49fe955859d4bbb8a9a2b7685ccec82,3146759,37038,4335922,PEW PEW PEW,$PEW,18.0
58,9244705,1578548918,0x42da33a736c7a5ddda8df2489492608d1571400106f3...,222,0x742ce596b4ddfbb3902a3a46949bf671c69b53317762...,92,0x002781a27becd321d831d815e3fa386944b329c8,0xdd4c48c0b24039969fc16d1cdf626eab821d3384,2850000000000000000,100000,...,NaN,NaN,deprecated,0x6b175474e89094c44da98b954eedeac495271d0f,4726093,50280,4335755,Dai Stablecoin,DAI,18.0
60,9244708,1578548950,0x34b5f6ef6307e1ca335f598803270d0dc3bc9ec72478...,224,0x9269af71d0864366edc51980faec2731b566a876c9eb...,134,0x002781a27becd321d831d815e3fa386944b329c8,0x9b984d5a03980d8dc0a24506c968465424c81dbe,2000000000000000000,100000,...,NaN,NaN,deprecated,0x6b175474e89094c44da98b954eedeac495271d0f,7528199,50280,4335752,Dai Stablecoin,DAI,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75034,9281595,1579037308,0x85b90d8583c19423acf8b1ee7ba8076b074929ef9ce2...,431,0x31777d2881f5ddafb21657fcb47cc96c3d861d6c5eb4...,64,0xffd1ac3e8818adcbe5c597ea076e8d3210b45df5,0xa19fcdad77c1f0fd184689aca88babcf68010347,10000000000000000000,100000,...,NaN,NaN,deprecated,0x6b175474e89094c44da98b954eedeac495271d0f,6258007,50280,4298954,Dai Stablecoin,DAI,18.0
75036,9281611,1579037479,0x1beae78b1a1892ba32e3b3d02d99e7e0579d5c1dee5c...,433,0x8678df3a27be6118bfc9666a672fc3b801ede9918bd4...,123,0xffd1ac3e8818adcbe5c597ea076e8d3210b45df5,0x865c2f85c9fea1c6ac7f53de07554d68cb92ed88,9500000000000000000,100000,...,NaN,NaN,deprecated,0x6b175474e89094c44da98b954eedeac495271d0f,7563655,50280,4298938,Dai Stablecoin,DAI,18.0
75038,9281617,1579037599,0x8626175fdff14a55cd4a36bdc07bd0d0e5437defd985...,435,0xfccbe781b1c5ed325ae546ed2a9fd7ff8816c73930ad...,71,0xffd1ac3e8818adcbe5c597ea076e8d3210b45df5,0x7d1675dc635959b26f61e03637599c12c7e84fae,9500000000000000000,100000,...,NaN,NaN,deprecated,0x6b175474e89094c44da98b954eedeac495271d0f,1875820,65280,4298932,Dai Stablecoin,DAI,18.0
75041,9291814,1579172586,0x777fddf8062c20bf0f71145535017e536a21dcc32d9f...,444,0xfa1faf4a75169fbea5e7abd3900c5d83a2364653b52a...,161,0xffd1ac3e8818adcbe5c597ea076e8d3210b45df5,0x1591b9056822c273252fcea32963223650db18e1,9500000000000000000,100000,...,NaN,NaN,deprecated,0x6b175474e89094c44da98b954eedeac495271d0f,7986910,50280,4288735,Dai Stablecoin,DAI,18.0


In [20]:
keep_tokenNames = ['ETH', 'Dai Stablecoin', 'Sai Stablecoin v1.0', 'Synth sUSD', 'USD Coin ']
rows_to_keep = df_filtered.tokenName.isin(keep_tokenNames)
df_a = df_filtered[rows_to_keep].copy()

In [21]:
df_a

,blockNumber,timeStamp,hash,nonce,blockHash,transactionIndex,from,to,value,gas,...,isError,txreceipt_status,input,contractAddress,cumulativeGasUsed,gasUsed,confirmations,tokenName,tokenSymbol,tokenDecimal
4,9272432,1578915412,0x9e49c1e0631b685ae2a2cbd1dc32e1e521729f92e661...,153,0x17f36b1282af804873b6fabcde9e1f221f4877403905...,83,0x00158a74921620b39e5c3afe4dca79feb2c2c143,0x322d58b9e75a6918f7e7849aee0ff09369977e08,4000000000000000000,100000,...,NaN,NaN,deprecated,0x6b175474e89094c44da98b954eedeac495271d0f,7217413,46008,4308028,Dai Stablecoin,DAI,18.0
52,9243147,1578528442,0x70518de1df557d61d92d3b7c1d5394fd264c634f54b4...,215,0xfc648bcb632d8f93255849f86be531415f25a6653fd9...,30,0x002781a27becd321d831d815e3fa386944b329c8,0xbd9cdcb6688b057080b1767261c5dc17a5804e82,23750000000000000000,100000,...,NaN,NaN,deprecated,0x6b175474e89094c44da98b954eedeac495271d0f,2992428,50292,4337313,Dai Stablecoin,DAI,18.0
58,9244705,1578548918,0x42da33a736c7a5ddda8df2489492608d1571400106f3...,222,0x742ce596b4ddfbb3902a3a46949bf671c69b53317762...,92,0x002781a27becd321d831d815e3fa386944b329c8,0xdd4c48c0b24039969fc16d1cdf626eab821d3384,2850000000000000000,100000,...,NaN,NaN,deprecated,0x6b175474e89094c44da98b954eedeac495271d0f,4726093,50280,4335755,Dai Stablecoin,DAI,18.0
60,9244708,1578548950,0x34b5f6ef6307e1ca335f598803270d0dc3bc9ec72478...,224,0x9269af71d0864366edc51980faec2731b566a876c9eb...,134,0x002781a27becd321d831d815e3fa386944b329c8,0x9b984d5a03980d8dc0a24506c968465424c81dbe,2000000000000000000,100000,...,NaN,NaN,deprecated,0x6b175474e89094c44da98b954eedeac495271d0f,7528199,50280,4335752,Dai Stablecoin,DAI,18.0
62,9245082,1578554240,0xcfa3ef5c22ec155c40cf55cdf80897b50d8b804c2ed3...,226,0x6edcaea6f1c825f842ed32b57d609fee087f1c270217...,20,0x002781a27becd321d831d815e3fa386944b329c8,0x11e4857bb9993a50c685a79afad4e6f65d518dda,2000000000000000000,100000,...,NaN,NaN,deprecated,0x6b175474e89094c44da98b954eedeac495271d0f,6928129,50280,4335378,Dai Stablecoin,DAI,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75034,9281595,1579037308,0x85b90d8583c19423acf8b1ee7ba8076b074929ef9ce2...,431,0x31777d2881f5ddafb21657fcb47cc96c3d861d6c5eb4...,64,0xffd1ac3e8818adcbe5c597ea076e8d3210b45df5,0xa19fcdad77c1f0fd184689aca88babcf68010347,10000000000000000000,100000,...,NaN,NaN,deprecated,0x6b175474e89094c44da98b954eedeac495271d0f,6258007,50280,4298954,Dai Stablecoin,DAI,18.0
75036,9281611,1579037479,0x1beae78b1a1892ba32e3b3d02d99e7e0579d5c1dee5c...,433,0x8678df3a27be6118bfc9666a672fc3b801ede9918bd4...,123,0xffd1ac3e8818adcbe5c597ea076e8d3210b45df5,0x865c2f85c9fea1c6ac7f53de07554d68cb92ed88,9500000000000000000,100000,...,NaN,NaN,deprecated,0x6b175474e89094c44da98b954eedeac495271d0f,7563655,50280,4298938,Dai Stablecoin,DAI,18.0
75038,9281617,1579037599,0x8626175fdff14a55cd4a36bdc07bd0d0e5437defd985...,435,0xfccbe781b1c5ed325ae546ed2a9fd7ff8816c73930ad...,71,0xffd1ac3e8818adcbe5c597ea076e8d3210b45df5,0x7d1675dc635959b26f61e03637599c12c7e84fae,9500000000000000000,100000,...,NaN,NaN,deprecated,0x6b175474e89094c44da98b954eedeac495271d0f,1875820,65280,4298932,Dai Stablecoin,DAI,18.0
75041,9291814,1579172586,0x777fddf8062c20bf0f71145535017e536a21dcc32d9f...,444,0xfa1faf4a75169fbea5e7abd3900c5d83a2364653b52a...,161,0xffd1ac3e8818adcbe5c597ea076e8d3210b45df5,0x1591b9056822c273252fcea32963223650db18e1,9500000000000000000,100000,...,NaN,NaN,deprecated,0x6b175474e89094c44da98b954eedeac495271d0f,7986910,50280,4288735,Dai Stablecoin,DAI,18.0


Take a look at the top 10 contributors by # of contributions.

In [22]:
df_a['from'].value_counts()[:10]

0x11111254369792b2ca5d084ab5eea397ca8fa48b    400
0x2a1530c4c41db0b0b2bb646cb5eb1a67b7158667     67
0x839395e20bbb182fa440d08f850e6c7a8f6f0780     65
0xe937e88591f641a689e2132a6f633d7b6b1a54c8     52
0xa4723a67376abd6ce94f5ea2e4bd37b5c5548804     48
0xe1a03133136ef1171bcb5fcd006496f22122d824     46
0xdcbeffbecce100cce9e4b153c4e15cb885643193     42
0x27445c691625f9b50292050bd36b9d008062991f     40
0x7901bb5d22b623e18d10e02ca67e4f50d0a49a18     37
0x0492b9170ec01d91f3aa721ff8ada42dd33fbbb8     36
Name: from, dtype: int64

## Numbers for comparison
from https://gitcoin.co/blog/gitcoin-grants-round-4/

* 143,642 in funds
* 5936 contributions were made to 230 open source Ethereum projects
* 1115 unique Ethereum community members

## Converting USD for Comparison with Blog Post
Jan 20 2020 Eth Value in USD: 166.90

Source: https://finance.yahoo.com/quote/ETH-USD/history?period1=1577923200&period2=1578614400&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true

In [24]:
df_a.loc[df_a.tokenName == 'ETH', 'value'].astype(float) / 10**18 * 166.90

334       66.760000
343        0.100140
350        0.041725
373        0.041725
387        0.041725
            ...    
69278    333.800000
73945    380.428075
74175      2.002800
74366      0.230080
74367      1.150400
Name: value, Length: 253, dtype: float64

In [48]:
def get_eth_to_usd(dt):
    """
    
    Get the eth_to_usd ratio for a given datetime.
    PLACEHOLDER
    """
    if dt == pd.to_datetime('01-20-2020'):
        return 166.90

dt = pd.to_datetime('01-20-2020')

def my_convert(row):

    if row.tokenName == 'ETH':
        eth_to_usd = get_eth_to_usd(dt)
        return float(row.value) / (10**18) * eth_to_usd
    else: # assume it's already pegged to USD
        return float(row.value) / (10**18)


In [49]:
df_a['usd'] = df_a.apply(my_convert, axis=1)

In [50]:
df_a.usd

4          4.00
52        23.75
58         2.85
60         2.00
62         2.00
          ...  
75034     10.00
75036      9.50
75038      9.50
75041      9.50
75044    500.00
Name: usd, Length: 6698, dtype: float64

In [51]:
df_a.usd.describe()

count     6698.000000
mean        65.174762
std        924.092308
min          0.000000
25%          1.000000
50%          4.750000
75%          9.975000
max      43581.931000
Name: usd, dtype: float64

In [52]:
df_a.usd.sum()

436540.55878770177

In [53]:
df_a['to_name'] = df_a['to'].map(address_to_name)

In [54]:
import datetime

In [55]:
df_a['dt'] = df_a.timeStamp.apply(datetime.datetime.fromtimestamp)

In [56]:
df_a['dt'].describe()

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


count                    6698
unique                   4133
top       2020-01-17 06:59:18
freq                       20
first     2020-01-06 00:50:33
last      2020-01-20 23:56:53
Name: dt, dtype: object

In [40]:
cols = ['blockNumber', 'from', 'to', 'value', 'tokenName', 'usd', 'to_name', 'dt']
df_a[df_a.to_name == 'Trinity - Research for migration of Ethereum 1.x to 2.0'].sort_values('usd')[cols]

,blockNumber,from,to,value,tokenName,usd,to_name,dt


In [58]:
df_a.groupby('to_name').usd.sum().sort_values(ascending=False)[:40]

to_name
Trinity - Ethereum 2.0 client                                  114085.575951
 Ether                                                          76632.329674
David Hoffman, Independent Content Creator                      35594.272030
Alice                                                           18013.166462
1inch.exchange on-chain DeFi aggregation protocol               15949.114752
Turbo-Geth                                                       9769.744504
Wizards of DApps podcast                                         9249.445019
Marketing DAO                                                    8815.934788
EtherLoans - undercollateralized loans, unstoppable DeFi         8068.003444
Zapper.fi                                                        7259.101766
Linkdrop                                                         6392.010389
TheBlockchainCircle                                              6283.765240
History as a Service / Untrusted DB                              594

In [60]:
df_a.to_name.unique()

array(['DeFi Saver', 'Enzypt.io', 'Tornado.cash',
       'Prysm by Prysmatic Labs', 'Uniswap',
       'DeFi Discussions Conference Support', 'MetaFactory',
       'Marketing DAO', 'Abridged', 'Voyager Media',
       'Fairdrop -  Secure, private & unstoppable file transfer for the free world.',
       'Trinity - Ethereum 2.0 client',
       '1inch.exchange on-chain DeFi aggregation protocol',
       'Commons Simulator: Level Up', 'Ideamarket.io', 'web3-react',
       'EthHub - Ethereum Information Hub', 'Zapper.fi',
       'Lighthouse: Ethereum 2.0 Client',
       '@antiprosynth 🦇 antiprosynthesis.eth ⟠ 🔊 Twitter account 🐬',
       'Chris Blec', 'wolfofethereum.eth', ' Ether',
       'Anti-Phishing Browser Plugin | PhishFort Protect', 'Dessert Dai',
       'David Hoffman, Independent Content Creator',
       'Decentralized Flexible Organization',
       'Solexplorer - Gitcoin Community Manager',
       'AvionDB: A Distributed, MongoDB-like Database (Fully compatible with Web2 & Web3)',


In [42]:
ex1 = df_a[df_a.to_name == 'Tornado.cash']

In [44]:
len(ex1)

229

In [45]:
ex1.usd.sum()

2844.67730826769

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=32e1581d-cef2-4771-a5bf-9bbfe7a3603c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>